In [16]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2

In [25]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in model.layers:
        print(layer.name, ': ', layer.trainable)
    # model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
      Args:
        base_model: keras model excluding top
        nb_classes: # of classes
      Returns:
        new keras model with last layer
    """
    
    x = base_model.output

    # GlobalAveragePooling2D converts the MxNxC tensor output into a 1xC tensor where C is the # of channels.
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) # new FC layer, random init
    # softmax function on the output to squeeze the values between [0,1]
    #predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
  Args:
    model: keras model
  """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    # model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])


def train():
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    
    nb_train_samples = 2000
    nb_classes = 2
    nb_val_samples = 800
    nb_epoch = 10
    batch_size = 16
    train_dir = 'data-pets/train'
    val_dir = 'data-pets/validation'
    output_model_file = 'inceptionv3.h5'

    # data prep
    train_datagen = ImageDataGenerator(
          preprocessing_function=preprocess_input, # zero-centers our image data
          rotation_range=30,
          width_shift_range=0.2,
          height_shift_range=0.2,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True
    )
    
    test_datagen = ImageDataGenerator(
          preprocessing_function=preprocess_input,
          rotation_range=30,
          width_shift_range=0.2,
          height_shift_range=0.2,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )

    # setup model
    # leave out the weights of the last fully connected layer
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    
    # transfer learning
    setup_to_transfer_learn(model, base_model)

    history_tl = model.fit_generator(
        train_generator,
        epochs=5,
        steps_per_epoch=nb_train_samples//batch_size,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
        #,
        #class_weight='auto'
    )
    
    # fine-tuning
    setup_to_finetune(model)
    
    history_ft = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
        #,
        #class_weight='auto'
    )
    
    model_json = model.to_json()
    with open("incep_filter.json", 'w') as json_file:
        json_file.write(model_json)
    
    model.save(output_model_file)


In [26]:
train()

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
input_12 :  False
conv2d_1035 :  False
batch_normalization_1035 :  False
activation_1035 :  False
conv2d_1036 :  False
batch_normalization_1036 :  False
activation_1036 :  False
conv2d_1037 :  False
batch_normalization_1037 :  False
activation_1037 :  False
max_pooling2d_45 :  False
conv2d_1038 :  False
batch_normalization_1038 :  False
activation_1038 :  False
conv2d_1039 :  False
batch_normalization_1039 :  False
activation_1039 :  False
max_pooling2d_46 :  False
conv2d_1043 :  False
batch_normalization_1043 :  False
activation_1043 :  False
conv2d_1041 :  False
conv2d_1044 :  False
batch_normalization_1041 :  False
batch_normalization_1044 :  False
activation_1041 :  False
activation_1044 :  False
average_pooling2d_100 :  False
conv2d_1040 :  False
conv2d_1042 :  False
conv2d_1045 :  False
conv2d_1046 :  False
batch_normalization_1040 :  False
batch_normalization_1042 :  False
batch_normalization_1045

Epoch 1/5
125/125 [==============================] - 81s - loss: 2.4970 - acc: 0.7080 - val_loss: 0.0938 - val_acc: 0.9700
Epoch 2/5
125/125 [==============================] - 77s - loss: 0.3367 - acc: 0.8705 - val_loss: 0.0954 - val_acc: 0.9613
Epoch 3/5
125/125 [==============================] - 77s - loss: 0.2603 - acc: 0.9115 - val_loss: 0.0820 - val_acc: 0.9663
Epoch 4/5
125/125 [==============================] - 77s - loss: 0.2101 - acc: 0.9130 - val_loss: 0.0950 - val_acc: 0.9637
Epoch 5/5
125/125 [==============================] - 77s - loss: 0.2245 - acc: 0.9155 - val_loss: 0.0945 - val_acc: 0.9575
Epoch 1/5
125/125 [==============================] - 117s - loss: 0.1236 - acc: 0.9510 - val_loss: 0.0841 - val_acc: 0.9650
Epoch 2/5
125/125 [==============================] - 112s - loss: 0.0893 - acc: 0.9650 - val_loss: 0.0632 - val_acc: 0.9725
Epoch 3/5
125/125 [==============================] - 112s - loss: 0.0948 - acc: 0.9600 - val_loss: 0.0663 - val_acc: 0.9762
Epoch 4/5
125